In [1]:
import cv2
import os
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np

from IPython.display import clear_output
import time
import PIL.Image
from io import BytesIO
import IPython.display

Using TensorFlow backend.


In [2]:
NOTEBOOK_DIR = !pwd
NOTEBOOK_DIR = NOTEBOOK_DIR[0]
ROOT_DIR = os.path.abspath(os.path.join(NOTEBOOK_DIR, os.pardir))
DATA_DIR = os.path.join(ROOT_DIR, 'data')

MNM_MODEL_DIR = os.path.join(ROOT_DIR, 'models/keras_sequential_model')

In [3]:
#Use 'jpeg' instead of 'png' (~5 times faster)
def array_to_image(a, fmt='jpeg'):
    #Create binary stream object
    f = BytesIO()

    #Convert array to binary stream object
    PIL.Image.fromarray(a).save(f, fmt)

    return IPython.display.Image(data=f.getvalue())

def get_frame(cam):
    # Capture frame-by-frame
    ret, frame = cam.read()

    #flip image for natural viewing
    frame = cv2.flip(frame, 1)

    return frame

In [4]:
# cam = cv2.VideoCapture(0)

# d = IPython.display.display("", display_id=1)
# d2 = IPython.display.display("", display_id=2)
# while True:
#     try:
#         t1 = time.time()
#         frame = get_frame(cam)
#         frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

#         im = array_to_image(frame)

#         d.update(im)

#         t2 = time.time()

#         s = f"""{int(1/(t2-t1))} FPS"""
#         d2.update( IPython.display.HTML(s) )
#     except KeyboardInterrupt:
#         print()
#         cam.release()
#         IPython.display.clear_output()
#         print ("Stream stopped")
#         break


In [6]:
cap = cv2.VideoCapture(0)
while True: 
    #Capture frame-by-frame
    __, frame = cap.read()
#     frame = get_frame(cap)
    print(frame)
    if frame is not None:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = array_to_image(frame)

        #Use MTCNN to detect faces
        result = detector.detect_faces(image)
        if result != []:

            for person in result:
                bounding_box = person['box']
                keypoints = person['keypoints']

                cv2.rectangle(frame,
                              (bounding_box[0], bounding_box[1]),
                              (bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),
                              (0,155,255),
                              2)

                cv2.circle(frame,(keypoints['left_eye']), 2, (0,155,255), 2)
                cv2.circle(frame,(keypoints['right_eye']), 2, (0,155,255), 2)
                cv2.circle(frame,(keypoints['nose']), 2, (0,155,255), 2)
                cv2.circle(frame,(keypoints['mouth_left']), 2, (0,155,255), 2)
                cv2.circle(frame,(keypoints['mouth_right']), 2, (0,155,255), 2)
        #display resulting frame
        cv2.imshow('frame',frame)
        if cv2.waitKey(1) &0xFF == ord('q'):
            break
#When everything's done, release capture
cap.release()
cv2.destroyAllWindows()

None
None
None
None
None
None


KeyboardInterrupt: 